### Conversion to PTL 

In [1]:
from read import text_recognizer
from model import Model
from utils import CTCLabelConverter
import torch
import cv2
from torch.utils.mobile_optimizer import optimize_for_mobile
import torchvision

C:\Users\Techistan\.conda\envs\urdu_ocr_utrnet\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file = open("UrduGlyphs.txt","r",encoding="utf-8")
content = file.readlines()
content = ''.join([str(elem).strip('\n') for elem in content])
content = content+" "
""" model configuration """
device = torch.device('cpu')
converter = CTCLabelConverter(content)
model = Model(num_class=len(converter.character), device=device)
model.to(device)

Model(
  (FeatureExtraction): HRNet_FeatureExtractor(
    (ConvNet): HRNet(
      (init_conv): Sequential(
        (0): Conv(
          (conv_bn): Sequential(
            (0): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
        (1): Conv(
          (conv_bn): Sequential(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (head): Sequential(
        (0): Conv(
          (conv_bn): Sequential(
            (0): Conv2d(960, 960, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
        (1): Conv2d(960, 32, ker

In [3]:
model.load_state_dict(torch.load('best_norm_ED.pth', map_location=device))

<All keys matched successfully>

In [4]:
model.eval()
from PIL import Image
img = Image.open("./img.png")
text_recognizer(img, model, converter, torch.device('cpu'))

'اب فقط دیکھتے ہی رہتے ہیں'

In [5]:
inp = torch.randn((1, 1, 32, 400), device=device)

model = torch.jit.trace(model.eval(), inp)

C:\Users\Techistan\.conda\envs\urdu_ocr_utrnet\lib\site-packages\torch\jit\_trace.py:985: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
With rtol=1e-05 and atol=1e-05, found 72767 element(s) (out of 72800) whose difference(s) exceeded the margin of error (including 0 nan comparisons). The greatest difference was 2.7878552675247192 (-3.2937562465667725 vs. -0.5059009790420532), which occurred at index (0, 62, 181).
  _module_class,


In [7]:
traced_script_module_optimized = optimize_for_mobile(model)
traced_script_module_optimized._save_for_lite_interpreter("FinalVersionMobileUnet.ptl")

#### For Testing 

for testing converting it to the jit model and inferring it

In [32]:
model.save("updated.pt")

In [8]:
model = torch.jit.load("FinalVersionMobileUnet.ptl")
model.eval()

RecursiveScriptModule(original_name=Model)

In [9]:
from PIL import Image
img = Image.open("./img.png")
text_recognizer(img, model, converter, torch.device('cpu'))

'اب فقط دیکھتے ہی رہتے ہیں'

## Conversion to ONNX 

In [35]:
from read import text_recognizer
from model import Model
from utils import CTCLabelConverter
import torch
import cv2
from torch.utils.mobile_optimizer import optimize_for_mobile
import torchvision

In [36]:
file = open("UrduGlyphs.txt","r",encoding="utf-8")
content = file.readlines()
content = ''.join([str(elem).strip('\n') for elem in content])
content = content+" "
""" model configuration """
device = torch.device('cpu')
converter = CTCLabelConverter(content)
model = Model(num_class=len(converter.character), device=device)
model.to(device)

Model(
  (FeatureExtraction): UNet_FeatureExtractor(
    (ConvNet): UNet(
      (inc): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
        )
      )
      (down1): Down(
        (maxpool_conv): Sequential(
          (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (1): DoubleConv(
            (double_conv): Sequential(
              (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [37]:
model.load_state_dict(torch.load('best_norm_ED.pth', map_location=device))

<All keys matched successfully>

In [38]:
inp = torch.randn((1, 1, 32, 400), requires_grad=False)
model.eval()
# Export the model
torch.onnx.export(model, inp, './unet.onnx',
              verbose=False, opset_version=11,
              do_constant_folding= True,
            export_params=True,
              input_names=["input"],
              output_names=["output"], 
#             dynamic_axes={"input": {0: "batch", 2: "height", 3: "width"}}
                 )
print("Model converted succesfully")
# onnx_program = torch.onnx.dynamo_export(model, inp)

Model converted succesfully
